# Identification

In the previous notebooks we simply used the conversion function to generate a system form a Transfer matrix $T$. In this notebook we will give more detail on the different strategies to generate systems, that are implemented in this library

In [ ]:
import numpy as np
from tvsclib.mixed_system import MixedSystem
import tvsclib.utils as utils

import timeit

## Identification using the shift invariance property


To constrct as system that has the given transfer matrix $T$ we first factor the Hankel matrices into the observability and reachability matrices.
If factor $H_k$ using the SVD into $H_k = U \Sigma V^\top$ we can get the state dimension by using the reduced SVD.

If we set 
$$\mathcal{O}_k = U \Sigma^{1/2} \quad\text{and}\quad \mathcal{R}_k = \Sigma^{1/2} V^\top$$
we obtian a balanced realisation.

Analogously we can get an input normal realzation by setting 
$$\mathcal{O}_k = U \Sigma \quad\text{and}\quad \mathcal{R}_k = V^\top$$
or an output normal realsation by setting
$$\mathcal{O}_k = U \quad\text{and}\quad \mathcal{R}_k = \Sigma V^\top$$

The matrices $C_k$ and $B_k$ can be be directly extracted form the matricies using the relationships
$$	\mathcal{O}_k = 
	\begin{bmatrix}
		C_k\\
		C_{k+1}A_k\\
		C_{k+2}A_{k+1}A_k\\
		\vdots
	\end{bmatrix}$$
    and
    $$
    	\mathcal{R}_{k+1} = \begin{bmatrix}
	 \cdots & A_{k}A_{k-1}B_{k-2} &A_{k}B_{k-1} &B_{k}
	\end{bmatrix}
    $$
    
To compute the matrix $A_k$ can use the fact that the the range of the Hankel matrices $H_k$ and $H_{k+1}$ can be realted.

If we remove the first row of the Hankel matrix 

$$H_k =
		\begin{bmatrix}
\cdots   & C_{k}A_{k-1}B_{k-2} & C_{k}B_{k-1} \\
\cdots   & C_{k+1}A_{k}A_{k-1}B_{k-2} & C_{k+1}A_{k}B_{k-1} \\
\cdots   & C_{k+2}C_{k+1}A_{k}A_{k-1}B_{k-2} & C_{k+2}A_{k+1}A_{k}B_{k-1} \\
 &\vdots &\vdots
\end{bmatrix}$$
we obtian the Matrix 
$$
H_k\!\uparrow		\begin{bmatrix}
\cdots   & C_{k+1}A_{k}A_{k-1}B_{k-2} & C_{k+1}A_{k}B_{k-1} \\
\cdots   & C_{k+2}C_{k+1}A_{k}A_{k-1}B_{k-2} & C_{k+2}A_{k+1}A_{k}B_{k-1} \\
 &\vdots &\vdots
\end{bmatrix}.
$$

This matrix can also be written as the product
$$H_k\!\uparrow = \mathcal{O_{k+1}} A_k \mathcal{R}_k$$

This is a consequnence of the shift invariance property.

The matrix $\mathcal{O_{k}}\!\uparrow = \mathcal{O_{k+1}} A_k$ can be obtained by removing the first block-row of $\mathcal{O_{k}}$.
Then we obtian $A_k$ by solving it using the Moore-Penrose Pseudoinverse $\mathcal{O_{k+1}}^\dagger$ of $\mathcal{O_{k+1}}$.
Finally 
$$A_k = \mathcal{O_{k+1}}^\dagger \mathcal{O_{k}}\!\uparrow$$

This algorithm and more details on the shift invariance property can be found in [[1](#Dewilde_1998)] on the pages 52-62.
This also gives an constructive proof that an arbritrary matrix can be represented by a time varying system.

A implementation of this algorithm is availabel in the module `SystemIdentificationSVD`
We can also give an $\epsilon$. This determeines if smaller singular values of the hankel matrices should be ignored.

In [ ]:
dims_in =  [15,15,15,15]
dims_out = [15,15,15,15]
rank = 5
T = np.random.rand(sum(dims_out),rank)@np.random.rand(rank,sum(dims_in))

In [ ]:
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator

TOs = ToeplitzOperator(T, dims_in, dims_out)
S = SystemIdentificationSVD(TOs,epsilon=1e-14)
system_a = MixedSystem(S)
print(system_a)

This algorithm explicitly computes and stores the Observability and Reachability matrices for all stages. This meanst, that the memory requirements can be prohibitivly for large systems.
The additional computation of the pseudoinverse also adds computational overhead.
To aleveiate these issues, a second implelemtation is provided.

## Identification without pseudoinverse

It is also possible to use an anglorithm that does not need an explicit computation of the Pseudoinverse.
For this we agian factor the Hankel Matrix into $H_k = U \Sigma V^\top$. 
We then set
$$\mathcal{O}_k = U \quad\text{and}\quad \mathcal{R}_k = \Sigma V^\top.$$

Based on this we agian imediatly get $C_k$ and $B_{k-1}$.
Additionally we get $\mathcal{O_{k}}\!\uparrow$.
Now we need to again compute $A_k$ by solving 
$$\mathcal{O_{k}}\!\uparrow = \mathcal{O_{k+1}} A_k.$$
If we multiply it form the right left with $\mathcal{O}_{k+1}^\top$ we get
$$\mathcal{O}_{k+1}^\top \mathcal{O_{k}}\!\uparrow = \mathcal{O}_{k+1}^\top\mathcal{O_{k+1}} A_k.$$
As the realization is output normal we have $\mathcal{O}_{k+1}^\top\mathcal{O_{k+1}}=1$
and therefore get
$$\mathcal{O}_{k+1}^\top \mathcal{O_{k}}\!\uparrow = A_k.$$

This results in an output normal system. To get an balanced or output normal system we can transform it using the singular values.
This can be done efficintly as the required transformation matrix is a diagonbal matrix.

A similar algorithm is also described in [[2](Chandrasekaran_2002)].


In [ ]:
import tvsclib.identification as ident
system_b = ident.identify(matrix_a, dims_in, dims_out,epsilon=1e-14)
print(system_a)

The identification in the identification module is the prevered implementation as it is faster and does not require additional memory to store large intermediate results.

In [ ]:
T = np.random.rand(500,500)
dims_in  = np.array(50*[10])
dims_out = np.array(50*[10])

In [ ]:
timeit.timeit(lambda:ident.identify(T, dims_in, dims_out,epsilon=1e-14), number=1)

In [ ]:
timeit.timeit(lambda:MixedSystem(SystemIdentificationSVD(
    ToeplitzOperator(T, dims_in, dims_out),epsilon=1e-14)), number=1)

## Further Notes

A third strategy to get an system represerntation form a matrix is described in [Masterarbeit]. Here we start by converting the Matrix into a trivial system with only one stage. These stages can be recursiveley split until we reach the desired input and output dimensions. 

### References
[1]:<a id='Dewilde_1998'></a> P. Dewilde and A.-J. van der Veen. Time-Varying Systems and Computations. Jan. 1, 1998. ISBN : 978-0-7923-8189-1. [DOI : 10.1007/978- 1- 4757-
2817-0](https://doi.org/10.1007/978-1-4757-2817-0)

[2]:<a id='Chandrasekaran_2002'></a>
S. Chandrasekaran, P. Dewilde, M. Gu, T. Pals, and A. J. van der Veen.
“Fast Stable Solver for Sequentially Semi-separable Linear Systems of Equa-
tions”. In: High Performance Computing — HiPC 2002. Ed. by S. Sahni, V. K.
Prasanna, and U. Shukla. Berlin, Heidelberg: Springer, 2002, pp. 545–554.
ISBN : 978-3-540-36265-4. [DOI : 10.1007/3-540-36265-7_51](https://doi.org/10.1007/3-540-36265-7_51)